# LeNet-5 in TF 

#### Load dependencies 

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

#### Load Data 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/",one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


#### Set hyperparameters

In [3]:
epochs = 20
batch_size = 128
display_progress = 40 # after this many batches, output progress to screen
wt_init = tf.contrib.layers.xavier_initializer() # weight initializer

#### Set parameters for each layer

In [4]:
# input layer: 
n_input = 784

# first convolutional layer: 
n_conv_1 = 32
k_conv_1 = 3 # k_size

# second convolutional layer: 
n_conv_2 = 64
k_conv_2 = 3

# max pooling layer:
pool_size = 2
mp_layer_dropout = 0.25

# dense layer: 
n_dense = 128
dense_layer_dropout = 0.5

# output layer: 
n_classes = 10

#### Define placeholders 

In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

#### Define layers 

In [6]:
# dense layer with ReLU activation:
def dense(x, W, b):
    z = tf.add(tf.matmul(x, W), b)
    a = tf.nn.relu(z)
    return a

# convolutional layer with ReLU activation:
def conv2d(x, W, b, stride_length=1):
    xW = tf.nn.conv2d(x, W, strides=[1, stride_length, stride_length, 1], padding='SAME')
    z = tf.nn.bias_add(xW, b)
    a = tf.nn.relu(z)
    return a

# max-pooling layer: 
def maxpooling2d(x, p_size):
    return tf.nn.max_pool(x, 
                          ksize=[1, p_size, p_size, 1], 
                          strides=[1, p_size, p_size, 1], 
                          padding='SAME')

#### Design neural network architecture 

In [7]:
def network(x, weights, biases, n_in, mp_psize, mp_dropout, dense_dropout):

    # reshape linear MNIST pixel input into square image: 
    square_dimensions = int(np.sqrt(n_in))
    square_x = tf.reshape(x, shape=[-1, square_dimensions, square_dimensions, 1])
    
    # convolutional and max-pooling layers:
    conv_1 = conv2d(square_x, weights['W_c1'], biases['b_c1'])
    conv_2 = conv2d(conv_1, weights['W_c2'], biases['b_c2'])
    pool_1 = maxpooling2d(conv_2, mp_psize)
    pool_1 = tf.nn.dropout(pool_1, 1-mp_dropout)
    
    # dense layer: 
    flat = tf.reshape(pool_1, [-1, weights['W_d1'].get_shape().as_list()[0]])
    dense_1 = dense(flat, weights['W_d1'], biases['b_d1'])
    dense_1 = tf.nn.dropout(dense_1, 1-dense_dropout)
    
    # output layer: 
    out_layer_z = tf.add(tf.matmul(dense_1, weights['W_out']), biases['b_out'])
    
    return out_layer_z

#### Define variable dictionaries 

In [8]:
# super awesome snippet to avoid a nasty error, 
# on second thought, dont do it, causes more unexpected errors
# tf.reset_default_graph()

# Define variable dictionaries 
bias_dict = {
    'b_c1': tf.Variable(tf.zeros([n_conv_1])),
    'b_c2': tf.Variable(tf.zeros([n_conv_2])),
    'b_d1': tf.Variable(tf.zeros([n_dense])),    
    'b_out': tf.Variable(tf.zeros([n_classes]))
}

full_square_length = np.sqrt(n_input)
pooled_square_length = int(full_square_length / pool_size)
dense_inputs = pooled_square_length**2 * n_conv_2

weight_dict = {
    'W_c1': tf.get_variable('W_c1', [k_conv_1, k_conv_1, 1, n_conv_1], initializer=wt_init),
    'W_c2': tf.get_variable('W_c2', [k_conv_2, k_conv_2, n_conv_1, n_conv_2], initializer=wt_init),
    'W_d1': tf.get_variable('W_d1', [dense_inputs, n_dense], initializer=wt_init),    
    'W_out': tf.get_variable('W_out', [n_dense, n_classes], initializer=wt_init)
}

#### Build Model 

In [9]:
predictions = network(x, weight_dict, bias_dict, n_input, pool_size, mp_layer_dropout, dense_layer_dropout)

#### Define loss and optimizer 

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



#### Define evaluation metrics 

In [11]:
correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy_pct = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))*100

#### Initialization Op 

In [12]:
initializer_op = tf.global_variables_initializer()

#### Train the network in session

In [13]:
with tf.Session() as session:
    session.run(initializer_op)
    
    print("Training for ", epochs, "epochs.")
    
    #loop over epochs:
    for epoch in range(epochs):
        
        avg_cost = 0.0
        avg_accuracy_pct = 0.0
        
        #loop over batches
        n_batches = int(mnist.train.num_examples / batch_size)
        for i in range(n_batches):
            
            # reassurance!
            if i % display_progress == 0:
                print("Step ", i+1, " of ", n_batches, " in epoch ", epoch+1, ".", sep='')
            
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            _, batch_cost, batch_acc = session.run([optimizer, cost, accuracy_pct],
                                                  feed_dict={x: batch_x, y: batch_y})
            
            avg_cost+= batch_cost / n_batches
            avg_accuracy_pct+= batch_acc / n_batches
        
        print("Epoch ", '%03d' % (epoch+1),
              ": cost =", '{:.3f}'.format(avg_cost),
              ", accuracy = ", '{:.2f}'.format(avg_accuracy_pct), "%",
                sep='')
        
    print("Training complete. Testing model.\n")

    test_cost = cost.eval({x: mnist.test.images, y: mnist.test.labels})
    test_accuracy_pct = accuracy_pct.eval({x: mnist.test.images, y: mnist.test.labels}) 

    print("Test Cost:", '{:.3f}'.format(test_cost))
    print("Test Accuracy: ", '{:.2f}'.format(test_accuracy_pct), "%", sep='')

Training for  20 epochs.
Step 1 of 429 in epoch 1.
Step 41 of 429 in epoch 1.
Step 81 of 429 in epoch 1.
Step 121 of 429 in epoch 1.
Step 161 of 429 in epoch 1.
Step 201 of 429 in epoch 1.
Step 241 of 429 in epoch 1.
Step 281 of 429 in epoch 1.
Step 321 of 429 in epoch 1.
Step 361 of 429 in epoch 1.
Step 401 of 429 in epoch 1.
Epoch 001: cost =0.246, accuracy = 92.68%
Step 1 of 429 in epoch 2.
Step 41 of 429 in epoch 2.
Step 81 of 429 in epoch 2.
Step 121 of 429 in epoch 2.
Step 161 of 429 in epoch 2.
Step 201 of 429 in epoch 2.
Step 241 of 429 in epoch 2.
Step 281 of 429 in epoch 2.
Step 321 of 429 in epoch 2.
Step 361 of 429 in epoch 2.
Step 401 of 429 in epoch 2.
Epoch 002: cost =0.089, accuracy = 97.33%
Step 1 of 429 in epoch 3.
Step 41 of 429 in epoch 3.
Step 81 of 429 in epoch 3.
Step 121 of 429 in epoch 3.
Step 161 of 429 in epoch 3.
Step 201 of 429 in epoch 3.
Step 241 of 429 in epoch 3.
Step 281 of 429 in epoch 3.
Step 321 of 429 in epoch 3.
Step 361 of 429 in epoch 3.
Step 40

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Conv2D_1 = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Relu, W_c2/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'Conv2D_1', defined at:
  File "c:\users\siddh\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\siddh\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\siddh\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File